In [1]:
import pandas as pd

In [36]:
pohang = pd.read_csv("../data/pohang_token.csv")

/tmp/ipykernel_5225/2407082767.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  pohang = pd.read_csv("../data/pohang_token.csv")


In [3]:
pohang.shape

(50296, 61)

In [ ]:
def drop_pretext(x):
    x = x.replace("[민원 유입 경로]: 모바일\n[사고발생지역]", "")
    x = x.replace("G:휴대폰(안전신문고 앱 제외) 또는 PC에 저장된 사진\n* C:안전신문고 앱으로 현장에서 촬영 후 바로 신고한 사진\n* S:안전신문고 앱으로 촬영 및 저장 후 당일 신고한 사진\n * 안전신문고 앱으로 촬영한 사진은 촬영 일시가 자동으로 표기되고, 위·변조 방지기능을 탑재\n\n\n[처리요청기관]:", "")
    x = x.replace("안전신문고 신고파일(사진) 촬영시간 및 경로 안내", "")
    return x

In [70]:
pohang['본문'] = pohang['본문'].apply(lambda x :drop_pretext(str(x)))

In [76]:
keyword_check('사진')['본문'][16]

'경상북도 포항시 남구 송도동 474-8\n장애인주차구역 침범\n\n*  *\n * (1/1) G: 2020/07/03 12:25:43\n* 포항시\n\n\n'

In [37]:
#분석에서 제거할 키워드를 추가하면 check_freq_tokens()함수에서 데이터 프레임 결과를 보여줄때 제거됨

stopwords = [
    'Too Long', '위반', '차량', '민원', '내용', '관련', '신고', '처리', '해당', '첨부파일', '확인', '문의',
    '다운로드', '요청', '제보', '발생', '경우', '도로', '업무', '주차', '불법', '답변', '사진', '촬영', '안전',
    '신문고', '과태료', '첨부', '저장', '안내', '부과', '갤러리', '불편', '정보', 
]

In [33]:
from collections import Counter  

def check_freq_tokens(x):
    tf = Counter()
    
    for token in x.tokens:
        tf.update(eval(token))
    tf_df = pd.DataFrame([tf.keys(), tf.values()]).T
    tf_df = tf_df[tf_df[0].apply(lambda x : len(x) > 1)] # 키워드의 길이가 2이상인 것만 표시
    tf_df = tf_df[tf_df[0].apply(lambda x : x not in stopwords)] # 키워드의 길이가 2이상인 것만 표시
    return tf_df.sort_values(1, ascending=False)

In [71]:
check_freq_tokens(pohang).head(50)

,0,1
497,사진,108941
498,촬영,74844
494,안전,71998
495,신문고,69067
934,정차,64348
936,과태료,52463
3,포항시,46353
499,안내,42277
550,첨부,41296
503,저장,38177


신고 관련된 민원이 많아보여서 체크

In [6]:
def keyword_check(x):
    lst_idx = []
    for idx, row in pohang.iterrows():
        if x in row.tokens:
            lst_idx.append(idx)
    return pohang.iloc[lst_idx, :]

'민원 유입 경로' 패턴으로 어느정도 처리 가능함 발견

In [44]:
import re

pattern = re.compile('민원 유입 경로')

normal_idx = pohang['본문'].apply(lambda x : pattern.search(str(x)) == None)

sum(normal_idx)


29680

In [9]:
pohang = pohang[normal_idx].reset_index(drop=True)

In [10]:
check_freq_tokens(pohang)

,0,1
511,사진,45169
137,신고,42318
512,첨부,34259
141,관련,34173
238,문의,33126
...,...,...
19652,사회면,1
19654,취수,1
19656,김창,1
19657,임계,1


In [49]:
pattern = re.compile('민원 유입 경로')

normal_idx = pohang['본문'].apply(lambda x : pattern.search(str(x)) != None)
pohang[normal_idx]

,Unnamed: 0,민원신청번호,민원접수번호,제목,본문,처리내용,처리이력,신청일,민원분야,민원발생지,...,MBR_BRTH,RRQST_PTTN_YN,ALL_LWRD_NM,MPTTN_YN,MPTTN_PTTN_RQST_NO,ED_PRCS_RLTN,T01_CMPL_INST_RCPT_SN,T02_CMPL_INST_RCPT_SN,세번이상종결처리관계,tokens
16,54,1AA-2007-0080740,2AA-2007-0087869,(SPP-2007-0302556)장애인주차구역 침범\n\n* 안전신문고 신고파일(사...,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 포항시 남구 송도동 474-8...,포항시청 노인장애인 복지과 김혜경 입니다.\n\n부과예정일 오류로 재답변 드립니다....,1. 저희 시 전자민원에 참여해 주셔서 감사합니다. \n\n신고주신차량은 2020....,2020-07-03 13:54:55,경찰,경상북도,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['민원', '유입', '경로', '모바일', '사고', '발생', '지역', '경..."
17,56,1AA-2006-0911420,2AA-2007-0026184,(SPP-2006-3003435)6월 2일에 신고했는데 아직 수리가 안되었네요. 철주가,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 포항시 북구 청하면 청계리 2...,해당 통신주에 대해 사업자가 정비완료 하였습니다.,1. 안녕하십니까! 공중케이블 정비 민원센터입니다. \n선생님께서 국민신문고를 통해...,2020-06-30 18:16:02,경찰,경상북도,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['민원', '유입', '경로', '모바일', '사고', '발생', '지역', '경..."
18,57,1AA-2007-0290957,2AA-2007-0296503,(SPP-2007-1002497)장애인주차구역 위반\n\n* 안전신문고 신고파일(사...,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 포항시 남구 연일읍 생지리 3...,포항시청 노인장애인 복지과 김혜경 입니다.\n\n죄송합니다 2020.07.20일 부...,1. 우리 시 전자민원에 참여해 주셔서 감사합니다.\n\n[신고주신 차량은 2020...,2020-07-10 15:30:12,경찰,경상북도,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['민원', '유입', '경로', '모바일', '사고', '발생', '지역', '경..."
33,78,1AA-2011-0009133,2AA-2011-0032671,(SPP-2011-0102501)차선변경시 깜빡이 켜지않음(지속적)\n\n* 안전신...,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 포항시 북구 청하면 소동리 산...,안녕하십니까 \n대구달서경찰서 교통관리계에 근무하는 행정관 김은주입니다 \n먼저 국...,안녕하십니까 \n대구달서경찰서 교통관리계에 근무하는 행정관 김은주입니다 \n먼저 국...,2020-11-01 15:22:12,경찰,경상북도,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Too Long']
38,83,1AA-2011-0427261,2AA-2011-0447386,(SPP-2011-1503535)어린이 공원에 팻말이 태풍때 떨어졌는데 아직도 나무에,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 포항시 남구 일월동 234-2...,1. 귀하께서 국민신문고를 통해 신청하신 민원(신청번호1AA-2011-0427261...,NaN,2020-11-15 14:44:56,행정·안전,경상북도,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['민원', '유입', '경로', '모바일', '사고', '발생', '지역', '경..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50021,72512,1AA-1906-121388,2AA-1906-151774,(SPP-1906-0402970)버스정류소 불법 주정차 신고입니다.\n\n* 안전신...,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 포항시 남구 오천읍 문덕리 4...,"1. 귀하께서 안전신문고 앱을 통해 (버스정류소, 횡단보도, 인도,소화전, 모퉁이)...",NaN,2019-06-04 19:24:54,경찰,경상북도,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['민원', '유입', '경로', '모바일', '사고', '발생', '지역', '경..."
50032,72536,1AA-1909-265157,2AA-1909-380764,(SPP-1909-1201710)수연면가 건물 2층 올라가는 현관문이 반사거울형식으로,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 포항시 남구 오천읍 세계리 8...,1. 평소 시정에 협조하여 주신데 대하여 감사드립니다. \n2. 귀하께서 국민신문고...,1. 평소 시정에 협조하여 주신데 대하여 감사드립니다.\n2. 귀하께서 국민신문고 ...,2019-09-12 16:11:58,기타,경상북도,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['민원', '유입', '경로', '모바일', '사고', '발생', '지역', '경..."
50035,72544,1AA-1904-300260,2AA-1904-372015,(SPP-1904-1701160)테일램프 스모그필름 불법튜닝 신고합니다\n\n* 안...,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 포항시 북구 죽도동 596-1...,1. 평소 자동차교통안전문화 발전의 관심에 깊이 감사드립니다.\n2. 먼저 국민신문...,NaN,2019-04-17 11:04:23,경찰,경상북도,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['민원', '유입', '경로', '모바일', '사고', '발생', '지역', '경..."
50040,72549,1AA-1906-827942,2AA-1906-892272,(SPP-1906-2600334)인도불법주정차\n\n* 안전신문고 신고파일(사진) ...,[민원 유입 경로]: 모바일\n[사고발생지역]경상북도 포항시 남구 지곡동 398\n...,"1. 귀하께서 안전신문고 앱을 통해 (버스정류소, 횡단보도, 인도,소화전, 모퉁이)...",NaN,2019-06-26 07:31:13,경찰,경상북도,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['민원', '유입', '경로', '모바일', '사고', '발생', '지역', '경..."


제목에 '생활불편신고'라는 말이 들어가면 단순 신고인 경우가 많음

In [11]:
pattern = re.compile('생활불편신고')

normal_idx = pohang['제목'].apply(lambda x : pattern.search(str(x)) == None)

sum(normal_idx)

19570

In [12]:
pohang = pohang[normal_idx].reset_index(drop=True)

In [35]:
check_freq_tokens(pohang).head(50)

,0,1
176,포항,13570
3,포항시,10504
82,경북,3713
1282,담당자,3643
4,남구,3341
264,북구,3199
629,사람,2971
72,생각,2896
1285,이첩,2856
1294,이송,2823
